# Generate Random numbers nPr with r between 2 values

In [1]:
#generate random permutations nPr = P(n,r) = n!/(n-r)!
import random
import itertools
import pandas as pd
import numpy as np

n = 12 #candidates (Meath 14, Dublin North 12, Dublin West 9)
r = 7  #preference cast
o = 10 #number of permutations to be generated
nlist = list(range(1, n+1)) 

randGenPer = random.sample(list(itertools.permutations(nlist, r)), o)

In [2]:
print(randGenPer)

[(6, 3, 11, 2, 9, 4, 10), (11, 10, 7, 4, 1, 5, 12), (2, 9, 10, 11, 7, 6, 4), (12, 7, 10, 4, 11, 9, 6), (10, 11, 4, 6, 7, 9, 1), (6, 2, 12, 11, 8, 3, 1), (1, 7, 9, 5, 12, 10, 3), (4, 6, 5, 10, 7, 12, 11), (12, 6, 1, 4, 10, 9, 8), (9, 3, 7, 4, 10, 1, 6)]


## Convert the permutation lists into a list of votes

In [3]:
#function to convert a permutation list into vote
def listToVote(listgen, num):
    voteList=[]
    for j in range(num):
        voteList.append(np.NaN)
    #print(voteList)

    for k in range(len(listgen)):
        voteList[listgen[k]-1] =k+1 
    return(voteList)
    

In [4]:
#convert each of the permutations generated into a vote
votesGen=[]
for row in randGenPer:
    #print(row)
    votesGen.append(listToVote(row,n))
print(votesGen)

[[nan, 4, 2, 6, nan, 1, nan, nan, 5, 7, 3, nan], [5, nan, nan, 4, 6, nan, 3, nan, nan, 2, 1, 7], [nan, 1, nan, 7, nan, 6, 5, nan, 2, 3, 4, nan], [nan, nan, nan, 4, nan, 7, 2, nan, 6, 3, 5, 1], [7, nan, nan, 3, nan, 4, 5, nan, 6, 1, 2, nan], [7, 2, 6, nan, nan, 1, nan, 5, nan, nan, 4, 3], [1, nan, 7, nan, 4, nan, 2, nan, 3, 6, nan, 5], [nan, nan, nan, 1, 3, 2, 5, nan, nan, 4, 7, 6], [3, nan, nan, 4, nan, 2, nan, 7, 6, 5, nan, 1], [6, nan, 2, 4, nan, 7, 3, nan, 1, 5, nan, nan]]


## Make data frame from list and write to file

In [5]:
#generate data frame from list
dfRandGenPer = pd.DataFrame(votesGen)
#drop all dublicate rows keeping last one
dfRandGenPer.drop_duplicates(keep='last', inplace=True)

In [13]:
# float_format='%.0f' to get rid of the decimal 1 not 1.0
#write to csv file
dfRandGenPer.to_csv('../data/randGenPer-'+str(n)+'P'+str(r)+'-r-'+str(o)+'Generated.csv', 
                    float_format='%.0f')
